In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.stats import pearsonr
from statsmodels.graphics.regressionplots import abline_plot

In [49]:
data = pd.read_csv('nhats_altered_subset.csv')

### Let's start creating some models to examine the relatioships between various measures of disability and some indicators of social well-being.

#### First, let's try a set of 6 independent variables:

hearing_aid_used: whether or not a hearing aid was used

pain_limits_activity: whether or not pain ever limited activity

sppb_score: a simple physical performance battery score, an indicator of basic physical strength, balance, and mobility

balance_score: the result of a physical test measuring balance

walk_score: the result of a physical test measuring ability to walk

go_outside_use_device: whether or not the individual ever uses a mobility device to go outside

#### And one dependent variable

go_out_enjoy: whether or not the individual ever goes out for enjoyment

##### We will subset our data, remove the observations with missing values, and then build the model with StatsModels.

In [13]:
# First we will subset our data because there is no need to work with the whole data frame
data_subset_1 = data[['hearing_aid_used', 'pain_limits_activity', 'sppb_score', 'balance_score', 
                      'walk_score', 'go_outside_use_device', 'go_out_enjoy']] 

# Next we need to get rid of incomplete observations
# Convert -1 (the encoding for a missing value) to nan and then drop them
data_subset_1 = data_subset_1.replace(-1, np.nan)
data_subset_1 = data_subset_1.dropna()

# Check that this has left us with a reasonable number of observations
# We should still have over 2,000 observations that are complete in all 7 fields
print(data_subset_1.shape)

(2381, 7)


In [22]:
# Specify the independent and dependent variables
X = data_subset_1[['hearing_aid_used', 'pain_limits_activity', 'sppb_score', 'balance_score', 
          'walk_score','go_outside_use_device']]
Y = data_subset_1['go_out_enjoy']

# Add a constant using StatsModels
X = sm.add_constant(X) 

# We will have to treat our values as floats for the model to build correctly
model_1 = sm.OLS(Y.astype(float), X.astype(float)).fit()
 
print_model_1 = model_1.summary()
print(print_model_1)

                            OLS Regression Results                            
Dep. Variable:           go_out_enjoy   R-squared:                       0.111
Model:                            OLS   Adj. R-squared:                  0.108
Method:                 Least Squares   F-statistic:                     49.25
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           2.93e-57
Time:                        10:54:22   Log-Likelihood:                -1232.1
No. Observations:                2381   AIC:                             2478.
Df Residuals:                    2374   BIC:                             2519.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     0.52

C:\Users\halli_000\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


##### In this model, it looks like we have three predictors that might be worth considering: 

1. The use of a hearing aid seems to make it substantially more likely that a person will go out for enoyment.
2. A higher score on the physical battery seems to make it slightly more likely.
3. A higher walk score seems to make it slightly more likely as well. 

All of these coefficients have P values that are significant at at least an alpha level of .05. The R squared value of this model is not very high, but don't forget what we're looking at: the reasons that an aging person does or does not go out for enjoymet. Human behaior is complicated, and this is likely to be an extremely over-determined variable. It may not be possible to build a single comprehensible model that accounts for a high percentage of variation in whether or not individuals go out.

##### Next, let's build a model to look at their interactions.

In [28]:
# For ease, we will call the StatsModels formula API using smf
# This does not change how the OLS model is created, but it lets us use R-like syntax for interactions
model_2 = smf.ols(formula='go_out_enjoy ~ walk_score * hearing_aid_used * sppb_score', 
                 data=data_subset_1).fit()

print(model_2.summary())

                            OLS Regression Results                            
Dep. Variable:           go_out_enjoy   R-squared:                       0.118
Model:                            OLS   Adj. R-squared:                  0.116
Method:                 Least Squares   F-statistic:                     45.41
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           1.18e-60
Time:                        11:14:20   Log-Likelihood:                -1222.1
No. Observations:                2381   AIC:                             2460.
Df Residuals:                    2373   BIC:                             2506.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

##### These results are really surprising!

Here, we see that almost every interaction makes it slightly *less* likely that an individual will ever go out for enjoyment, even though all the predictors we focused on make it more likely individually. There are a lot of things that could be goig on, but all of them would require further study to isolate. For example, the most negative coefficiet is for the interaction between a high walk score and use of a hearing aid. Is it possible that individuals with a higher walking score tend to be younger, and more recent adopters of their hearing aids? Is the length of time an individual has used a hearing aid important? These would be great questions for a follow-up study.

#### Next, let's try the same 6 independent variables and a new dependent variable

num_in_soc_network: the number of people in an individual's social network

In [24]:
# We will re-subset our data because we will need to drop observations that are missing num_in_soc_net
# But we no longer care if a row is missing go_out_enjoy
# We don't want to end up with fewer observations than we could have
data_subset_2 = data[['hearing_aid_used', 'pain_limits_activity', 'sppb_score', 'balance_score', 
                      'walk_score', 'go_outside_use_device', 'num_in_soc_net']] 

# Next we need to get rid of incomplete observations
# Convert -1 (the encoding for a missing value) to nan and then drop them
data_subset_2 = data_subset_2.replace(-1, np.nan)
data_subset_2 = data_subset_2.dropna()

# Check that this has left us with a reasonable number of observations
# We should still have over 2,000 observations that are complete in all 7 fields
print(data_subset_2.shape)

(2289, 7)


In [26]:
# As before, specify the independent and dependent variables
X2 = data_subset_2[['hearing_aid_used', 'pain_limits_activity', 'sppb_score', 'balance_score', 
          'walk_score','go_outside_use_device']]
Y2 = data_subset_2['num_in_soc_net']

# Add a constant using StatsModels
X2 = sm.add_constant(X2) 

# We will have to treat our values as floats for the model to build correctly
model_3 = sm.OLS(Y2.astype(float), X2.astype(float)).fit()
 
print_model_3 = model_3.summary()
print(print_model_3)

                            OLS Regression Results                            
Dep. Variable:         num_in_soc_net   R-squared:                       0.028
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     10.97
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           4.41e-12
Time:                        10:58:33   Log-Likelihood:                -3915.2
No. Observations:                2289   AIC:                             7844.
Df Residuals:                    2282   BIC:                             7884.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                     1.75

C:\Users\halli_000\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


##### General Observations

The R-squared value for this model is extremely low, to the extent that it calls the utility of this model into question (even given some willingness to accept low R-squared values in this domain).

##### In this model, it looks like we have four predictors that might be worth considering: 

1. Pain that sometimes limits activity seems as though it may be associated with a bigger social network! The p-value for this coefficient is the lowest of the predictors listed here, but it is still significant at the alpha=0.05 level.
2. A higher balance score may be associated with a bigger social network.
3. A higher walk score seems to be associated with a bigger social network. 
4. The use of a device to go outside may be associated with a bigger social network.

Should we be surprised that use of an assistive device to go outside was positively correlated with a bigger social network, and in fact has the highest positive coefficient in this model? Maybe not! This may indicate that assistive devices are doing what they are supposed to do, and ameliorate the effect of mobility-related disabilities.

##### Next, let's build a model to look at their interactions.

In [27]:
# We will again use the StatsModels formula API because the syntax is simpler
model_4 = smf.ols(formula='num_in_soc_net ~ pain_limits_activity * balance_score * walk_score * go_outside_use_device', 
                 data=data_subset_2).fit()

print(model_4.summary())

                            OLS Regression Results                            
Dep. Variable:         num_in_soc_net   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.025
Method:                 Least Squares   F-statistic:                     4.845
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           2.19e-09
Time:                        11:13:47   Log-Likelihood:                -3911.7
No. Observations:                2289   AIC:                             7855.
Df Residuals:                    2273   BIC:                             7947.
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                                                                          coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

##### This is a lot of interactions to look at, but none of them approach significance.

In fact, now that we've included terms for interactions, none of the predictors we chose is significant individually, either.

#### Now let's try the same 6 independent variables and a new dependent variable

age_you_feel: the age a person feels mentally and emotionally

In [29]:
# Just like before we will re-subset the data to ensure we have as many complete observations as possible
data_subset_3 = data[['hearing_aid_used', 'pain_limits_activity', 'sppb_score', 'balance_score', 
                      'walk_score', 'go_outside_use_device', 'age_you_feel']] 

# Next we need to get rid of incomplete observations
# Convert -1 (the encoding for a missing value) to nan and then drop them
data_subset_3 = data_subset_3.replace(-1, np.nan)
data_subset_3 = data_subset_3.dropna()

# Check that this has left us with a reasonable number of observations
# We should still have over 2,000 observations that are complete in all 7 fields
print(data_subset_3.shape)

(2195, 7)


In [30]:
# As before, specify the independent and dependent variables
X3 = data_subset_3[['hearing_aid_used', 'pain_limits_activity', 'sppb_score', 'balance_score', 
          'walk_score','go_outside_use_device']]
Y3 = data_subset_3['age_you_feel']

# Add a constant using StatsModels
X3 = sm.add_constant(X3) 

# We will have to treat our values as floats for the model to build correctly
model_5 = sm.OLS(Y3.astype(float), X3.astype(float)).fit()
 
print_model_5 = model_5.summary()
print(print_model_5)

                            OLS Regression Results                            
Dep. Variable:           age_you_feel   R-squared:                       0.081
Model:                            OLS   Adj. R-squared:                  0.079
Method:                 Least Squares   F-statistic:                     32.24
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           2.28e-37
Time:                        11:33:21   Log-Likelihood:                -8851.5
No. Observations:                2195   AIC:                         1.772e+04
Df Residuals:                    2188   BIC:                         1.776e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
const                    73.60

C:\Users\halli_000\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


##### In this model, it looks like we have two predictors that might be worth considering: 

1. The use of a hearing aid seems to make individuals feel multiple years older than they might otherwise feel.
2. A higher score on the physical battery seems to make individuals feel a bit younger.

Both of these are significant at the alpha=0.01 level and the coefficients are fairly large. Let's take a second to consider the hearing aid, spoecifically. Isn't it interesting that it makes users feel older? Intuitively, that seems to make sense, but recall that hearing aid use was positively associated with going out for enjoymeny. (It was also positively associated with other markers of social wellbeing in models not illustrated here... keep experimenting with this data!) 

##### Now let's build a model to look at the interaction.

In [31]:
# We will continue to use the StatsModels formula API because the syntax is simpler
model_6 = smf.ols(formula='age_you_feel ~ hearing_aid_used * sppb_score', 
                 data=data_subset_3).fit()

print(model_6.summary())

                            OLS Regression Results                            
Dep. Variable:           age_you_feel   R-squared:                       0.079
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     62.46
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           9.56e-39
Time:                        11:40:26   Log-Likelihood:                -8854.4
No. Observations:                2195   AIC:                         1.772e+04
Df Residuals:                    2191   BIC:                         1.774e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept         

##### It doesn't look like this approach adds anything to our understanding.

The interaction is not significant, but considering it has not caused a major change on the individual role of the two variables.

### Let's create some models to examine the relatioships between some of the indicators of social well-being that we've considered so far.

#### Let's try a set of 3 independent variables:

group_activity: whether or not the individual ever participates in group activities.

go_out_enjoy: whether or not the individual ever goes out for enjoyment

num_in_soc_net: the number of people in the individual's social network

#### And one dependent variable:

age_you_feel: the age a person feels mentally and emotionally

##### We will subset our data, remove the observations with missing values, and then build the model with StatsModels.

In [50]:
# Unfortunately our data is not perfectly clean, so we will have to recode a few values
data = data.replace(['-7 RF'], ['-1'])
data[['group_activity']] = data[['group_activity']].apply(pd.to_numeric) 

# Check that this has worked
data.group_activity.unique()

In [52]:
# Just like before we will re-subset the data to ensure we have as many complete observations as possible
data_subset_4 = data[['group_activity', 'go_out_enjoy', 'num_in_soc_net', 'age_you_feel']] 

# Next we need to get rid of incomplete observations
# Convert -1 (the encoding for a missing value) to nan and then drop them
data_subset_4 = data_subset_4.replace(-1, np.nan)
data_subset_4 = data_subset_4.dropna()

# Check that this has left us with a reasonable number of observations
# In this case we have almost 4.5k observations left. Great!
print(data_subset_4.shape)

(4456, 4)


In [53]:
# As before, specify the independent and dependent variables
X4 = data_subset_4[['group_activity', 'go_out_enjoy', 'num_in_soc_net']]
Y4 = data_subset_4['age_you_feel']

# Add a constant using StatsModels
X4 = sm.add_constant(X4) 

# We will have to treat our values as floats for the model to build correctly
model_6 = sm.OLS(Y4.astype(float), X4.astype(float)).fit()
 
print_model_6 = model_6.summary()
print(print_model_6)

                            OLS Regression Results                            
Dep. Variable:           age_you_feel   R-squared:                       0.012
Model:                            OLS   Adj. R-squared:                  0.011
Method:                 Least Squares   F-statistic:                     17.44
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           2.98e-11
Time:                        11:58:55   Log-Likelihood:                -18322.
No. Observations:                4456   AIC:                         3.665e+04
Df Residuals:                    4452   BIC:                         3.668e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const             70.5476      0.572    123.

C:\Users\halli_000\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


##### In this model, it looks like we only have one predictor that might be worth considering: 

1. Going out for enjoyment is associated with feeling substantially younger. Over three and a half years!

##### Again, however, we have an extremely low R-squared value. So this model may not be indicitive of much.

##### Next let's try a slightly different view of these same indicators.

In [54]:
# As before, specify the independent and dependent variables
X5 = data_subset_4[['group_activity', 'go_out_enjoy', 'age_you_feel']]
Y5 = data_subset_4['num_in_soc_net']

# Add a constant using StatsModels
X5 = sm.add_constant(X5) 

# We will have to treat our values as floats for the model to build correctly
model_7 = sm.OLS(Y5.astype(float), X5.astype(float)).fit()
 
print_model_7 = model_7.summary()
print(print_model_7)

                            OLS Regression Results                            
Dep. Variable:         num_in_soc_net   R-squared:                       0.031
Model:                            OLS   Adj. R-squared:                  0.030
Method:                 Least Squares   F-statistic:                     47.12
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           5.68e-30
Time:                        12:02:48   Log-Likelihood:                -7550.3
No. Observations:                4456   AIC:                         1.511e+04
Df Residuals:                    4452   BIC:                         1.513e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
const              1.8675      0.103     18.

C:\Users\halli_000\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


##### In this model, we may have two predictors worth considering, but neither are surprising: 

1. Going out for enjoyment is associated with a slightly bigger social network.
2. Engaging in group activities is associated with a slightly bigger social network.

Both of these are highly significant, which makes a lot of intuitive sense.

##### In (many) other models not illustrated here it was generally found that the size of a social network was difficult to predict, especially using indicators of disability

The relationships between different indicators of social engagement and well-being might be interesting in their own right, but they also may shed some light on this. It is possible taht understanding the relationships of these other variables to various indicators of disability, and then to each other, might helop build a model to effectively predict the size of a social network using disability. This is a great avenue for future study.

### Finally, let's create some models that help us understand the relationships between a few measures of disability, a few indicators of social engagement, and a general indicator of psychological well-being.

#### Let's try another set of 6 independent variables:

group_activity: whether or not the individual ever participates in group activities.

go_out_enjoy: whether or not the individual ever goes out for enjoyment

num_in_soc_net: the number of people in the individual's social network

hearing_aid_used: whether or not a hearing aid was used

sppb_score: a simple physical performance battery score, an indicator of basic physical strength, balance, and mobility

walk_score: the result of a physical test measuring ability to walk

#### And one dependent variable:

life_has_meaning: whether or not an individual feels that life has meaning

##### We will subset our data, remove the observations with missing values, and then build the model with StatsModels.

In [55]:
# Just like before we will re-subset the data to ensure we have as many complete observations as possible
data_subset_6 = data[['group_activity', 'go_out_enjoy', 'num_in_soc_net', 'hearing_aid_used',
                     'sppb_score', 'walk_score', 'life_has_meaning']] 

# Next we need to get rid of incomplete observations
# Convert -1 (the encoding for a missing value) to nan and then drop them
data_subset_6 = data_subset_6.replace(-1, np.nan)
data_subset_6 = data_subset_6.dropna()

# Check that this has left us with a reasonable number of observations
# In this case we have over 4k observations left
print(data_subset_6.shape)

(4147, 7)


In [56]:
# As before, specify the independent and dependent variables
X6 = data_subset_6[['group_activity', 'go_out_enjoy', 'num_in_soc_net', 'hearing_aid_used',
                   'sppb_score', 'walk_score']]
Y6 = data_subset_6['life_has_meaning']

# Add a constant using StatsModels
X6 = sm.add_constant(X6)

# We will have to treat our values as floats for the model to build correctly
model_8 = sm.OLS(Y6.astype(float), X6.astype(float)).fit()
 
print_model_8 = model_8.summary()
print(print_model_8)

                            OLS Regression Results                            
Dep. Variable:       life_has_meaning   R-squared:                       0.061
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     44.94
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           1.52e-53
Time:                        12:17:53   Log-Likelihood:                -2510.4
No. Observations:                4147   AIC:                             5035.
Df Residuals:                    4140   BIC:                             5079.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
const                1.4844      0.020  

C:\Users\halli_000\Anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


##### In this model, we may have five predictors worth considering: 

NOTE: life_has_meaning has three possible values; 1 (agree a lot), 2 (agree a little), 3 (agree not at all)

1. Engaing in group activities is associated with a higher liklihood of believing that life has meaning.
2. Going out for enjoyment is also associated with a higher liklihood of believing that life has meaning.
3. So is having a larger social network.
4. Using a hearing aid is associated with a lower likelihood of believing that life has meaning. This is technically not significant even at an alpha=0.05 level but it is extremely close; in my opiion, it warrants further attention.
5. A high sppb score is associated with a higher liklihood of believing that life has meaning.

##### This gives us a lot of interesting interactions to consider, but let's just try looking at a few of them for now.

We will examine the potential interactions between go_out_enjoy (the predictor with the highest absolute coefficient), group_activity, and sppb_score.

In [62]:
# We will continue to use the StatsModels formula API because the syntax is simpler
model_9 = smf.ols(formula='life_has_meaning ~ group_activity * sppb_score * go_out_enjoy', 
                 data=data_subset_6).fit()

print(model_9.summary())

                            OLS Regression Results                            
Dep. Variable:       life_has_meaning   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     32.00
Date:                Wed, 11 Dec 2019   Prob (F-statistic):           1.53e-43
Time:                        13:03:56   Log-Likelihood:                -2531.9
No. Observations:                4147   AIC:                             5080.
Df Residuals:                    4139   BIC:                             5130.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

##### Only one of the interactions in this new model are significant, but two of them are close.

The interaction between engaging in group activities and a high sppb score has a P-value of 0.55, and indicates that this interaction may be associated with a slightly higher liklihood of believing that life has meaning.

The interaction between engaging in group activities and going out for enjoyment has the same P-value, but may be associated with a larger increase in liklihood of believing that life has meaning.

The interaction between all three predictors considered in this model is (barely) significant at an alpha=0.05 level, but surprisingly, it's associated wih a slightly lower liklihood of believing that life has meaning!